In [1]:
import os
import math
import datetime
import numpy as np
import pandas as pd


import time
import os.path

## step 1 obtain OSM network

In [2]:
"Get the OSM Network"
network_type = 'auto'
import osm2gmns as og

net = og.getNetFromOSMFile('tmc.osm',network_type=(network_type), default_lanes=True, default_speed=True)

og.consolidateComplexIntersections(net)
og.outputNetToCSV(net, output_folder=os.getcwd())

osm2gmns, version 0.4.2


## step 2 Convert tmc Data into node and link

In [2]:
"Convert TMC Data into GMNS Format"
tmc = pd.read_csv('TMC_Identification.csv')
tmc = tmc.drop_duplicates(subset=['direction','road_order']).sort_values(by=['direction','road_order'])
tmc = tmc.reset_index()
tmc = tmc.drop(['index'], 1)

In [3]:

'''build node.csv'''
print('converting tmc data into gmns format...')
p=1

node_tmc = pd.DataFrame()
node_tmc['name'] = None
node_tmc['x_coord'] = None
node_tmc['y_coord'] = None
node_tmc['z_coord'] = None
node_tmc['node_type'] = None
node_tmc['ctrl_type'] = None
node_tmc['zone_id'] = None
node_tmc['geometry'] = None

for i in range(0,len(tmc)-1):
    if tmc.loc[i+1,'road_order'] > tmc.loc[i,'road_order']:
        node_tmc = node_tmc.append({'name': tmc.loc[i,'tmc'],\
                                    'x_coord': tmc.loc[i,'start_longitude'], \
                                    'y_coord': tmc.loc[i,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) +")"}, ignore_index=True)
    else:
        node_tmc = node_tmc.append({'name': tmc.loc[i,'tmc'],\
                                    'x_coord': tmc.loc[i,'start_longitude'], \
                                    'y_coord': tmc.loc[i,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) +")"}, ignore_index=True)
        node_tmc = node_tmc.append({'name': tmc.loc[i,'tmc']+'END',\
                                    'x_coord': tmc.loc[i,'end_longitude'], \
                                    'y_coord': tmc.loc[i,'end_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_end',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'end_longitude'].astype(str) + " " + tmc.loc[i,'end_latitude'].astype(str) +")"}, ignore_index=True)

    if i > p/10 * len(tmc): 
        print(str(p*10)+"%"+' nodes completed!')
        p = p + 1

node_tmc = node_tmc.append({'name': tmc.loc[i+1,'tmc'],\
                                    'x_coord': tmc.loc[i+1,'start_longitude'], \
                                    'y_coord': tmc.loc[i+1,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i+1,'start_longitude'].astype(str) + " " + tmc.loc[i+1,'start_latitude'].astype(str) +")"}, ignore_index=True)

node_tmc = node_tmc.append({'name': tmc.loc[i+1,'tmc']+'END',\
                                    'x_coord': tmc.loc[i+1,'end_longitude'], \
                                    'y_coord': tmc.loc[i+1,'end_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_end',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i+1,'end_longitude'].astype(str) + " " + tmc.loc[i+1,'end_latitude'].astype(str) +")"}, ignore_index=True)

node_tmc.index.name = 'node_id'

node_tmc.index += 100000001 #index from 0

node_tmc.to_csv('node_tmc.csv')
print('node_tmc.csv (' + str(len(node_tmc)) + ' nodes' + ') generated!')

converting tmc data into gmns format...
10% nodes completed!
20% nodes completed!
30% nodes completed!
40% nodes completed!
50% nodes completed!
60% nodes completed!
70% nodes completed!
80% nodes completed!
90% nodes completed!
node_tmc.csv (64 nodes) generated!


In [4]:
node_tmc

,name,x_coord,y_coord,z_coord,node_type,ctrl_type,zone_id,geometry
node_id,,,,,,,,
100000001,110N05711,-77.3715,39.0144,None,tmc_start,None,None,POINT (-77.3715 39.0144)
100000002,110-05710,-77.3709,39.0141,None,tmc_start,None,None,POINT (-77.3709 39.0141)
100000003,110N05710,-77.3684,39.0127,None,tmc_start,None,None,POINT (-77.3684 39.0127)
100000004,110-05709,-77.3605,39.0086,None,tmc_start,None,None,POINT (-77.3605 39.0086)
100000005,110-05708,-77.3502,39.0028,None,tmc_start,None,None,POINT (-77.3502 39.0028)
...,...,...,...,...,...,...,...,...
100000060,110+05710,-77.3500,39.0030,None,tmc_start,None,None,POINT (-77.35 39.003)
100000061,110P05710,-77.3609,39.0090,None,tmc_start,None,None,POINT (-77.3609 39.009)
100000062,110+05711,-77.3673,39.0125,None,tmc_start,None,None,POINT (-77.3673 39.0125)


In [5]:
'''build link_tmc.csv'''
p = 1
link_tmc = pd.DataFrame()
link_tmc['name'] = None
link_tmc['corridor_id'] = None
link_tmc['corridor_link_order'] = None
link_tmc['from_node_id'] = None
link_tmc['to_node_id'] = None
link_tmc['directed'] = None
link_tmc['geometry_id'] = None
link_tmc['geometry'] = None
link_tmc['dir_flag'] = None
link_tmc['length'] = None
link_tmc['grade'] = None
link_tmc['facility_type'] = None
link_tmc['capacity'] = None
link_tmc['free_speed'] = None
link_tmc['lanes'] = None

for i in range(0,len(tmc)):
    link_tmc = link_tmc.append({'name': tmc.loc[i,'tmc'],\
                                'corridor_id': tmc.loc[i,'road']+'_'+tmc.loc[i,'direction'],\
                                'corridor_link_order' : tmc.loc[i,'road_order'],\
                                'from_node_id': node_tmc[(node_tmc['x_coord']==tmc.loc[i,'start_longitude']) & (node_tmc['y_coord']==tmc.loc[i,'start_latitude'])].index.values[0], \
                                'to_node_id': node_tmc[(node_tmc['x_coord']==tmc.loc[i,'end_longitude']) & (node_tmc['y_coord']==tmc.loc[i,'end_latitude'])].index.values[0],\
                                'directed': 1,\
                                'geometry_id': None,\
                                'geometry': "LINESTRING (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) + "," +\
                                    tmc.loc[i,'end_longitude'].astype(str) +" "+ tmc.loc[i,'end_latitude'].astype(str) + ")",\
                                'dir_flag': 1,\
                                'length': tmc.loc[i,'miles'],\
                                'grade': None,\
                                'facility_type': 'interstate' if tmc.loc[i,'road'][0] == 'I'else None ,\
                                'capacity':None,\
                                'free_speed':None,\
                                'lanes': None}, ignore_index=True)

    if i > p/10 * len(tmc): 
        print(str(p*10)+"%"+' links completed!')
        p = p + 1
        
link_tmc.index.name = 'link_id'
link_tmc.index += 100000001


link_tmc.to_csv('link_tmc.csv')

print('link_tmc.csv (' + str(len(link_tmc)) + ' links' + ') generated!')

10% links completed!
20% links completed!
30% links completed!
40% links completed!
50% links completed!
60% links completed!
70% links completed!
80% links completed!
90% links completed!
link_tmc.csv (62 links) generated!


In [6]:
link_tmc

,name,corridor_id,corridor_link_order,from_node_id,to_node_id,directed,geometry_id,geometry,dir_flag,length,grade,facility_type,capacity,free_speed,lanes
link_id,,,,,,,,,,,,,,,
100000001,110N05711,VA-7_EASTBOUND,38.0,100000001,100000002,1,None,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",1,0.038620,None,None,None,None,None
100000002,110-05710,VA-7_EASTBOUND,39.0,100000002,100000003,1,None,"LINESTRING (-77.3709 39.0141,-77.3684 39.0127)",1,0.164895,None,None,None,None,None
100000003,110N05710,VA-7_EASTBOUND,40.0,100000003,100000004,1,None,"LINESTRING (-77.3684 39.0127,-77.3605 39.0086)",1,0.512158,None,None,None,None,None
100000004,110-05709,VA-7_EASTBOUND,41.0,100000004,100000005,1,None,"LINESTRING (-77.3605 39.0086,-77.3502 39.0028)",1,0.682589,None,None,None,None,None
100000005,110-05708,VA-7_EASTBOUND,42.0,100000005,100000006,1,None,"LINESTRING (-77.3502 39.0028,-77.3455 39.0005)",1,0.296731,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100000058,110+05709,VA-7_WESTBOUND,45.0,100000059,100000060,1,None,"LINESTRING (-77.346 39.001,-77.35 39.003)",1,0.254799,None,None,None,None,None
100000059,110+05710,VA-7_WESTBOUND,46.0,100000060,100000061,1,None,"LINESTRING (-77.35 39.003,-77.3609 39.009)",1,0.718593,None,None,None,None,None
100000060,110P05710,VA-7_WESTBOUND,47.0,100000061,100000062,1,None,"LINESTRING (-77.3609 39.009,-77.3673 39.0125)",1,0.421999,None,None,None,None,None


In [7]:
## reading by detid
reading = pd.read_csv('Reading.csv')
reading = reading.loc[0:5000]


reading_dict = {}
gp = reading.groupby('tmc_code')
for key, form in gp:
    reading_dict[key] = {
        'measurement_tstamp':form['measurement_tstamp'].tolist(),
        'speed':form['speed'].tolist()
        }


In [8]:
'''build measurement_tmc.csv''' 

measurement_tmc = pd.DataFrame()
measurement_tmc['link_id_tmc'] = None
measurement_tmc['corridor_id'] = None
measurement_tmc['corridor_link_order'] = None
measurement_tmc['from_node_id'] = None
measurement_tmc['to_node_id'] = None
measurement_tmc['time_period'] = None
measurement_tmc['date'] = None
measurement_tmc['geometry'] = None
measurement_tmc['volume'] = None
measurement_tmc['travel_time'] = None
measurement_tmc['speed'] = None
measurement_tmc['reference_speed'] = None
measurement_tmc['density'] = None
measurement_tmc['queue'] = None
measurement_tmc['notes'] = None


k=0
p=1
measurement_tmc_dict = {}
for i in link_tmc.index:
    try:
        reading_dict_selected = reading_dict[link_tmc['name'][i]]
        for j in range(0,len(reading_dict_selected['measurement_tstamp'])):
            measurement_tmc_dict[k] = {'link_id_tmc': i,\
                                            'corridor_id': link_tmc['corridor_id'][i],\
                                            'corridor_link_order' : link_tmc['corridor_link_order'][i],\
                                            'from_node_id': link_tmc.loc[i,'from_node_id'], \
                                            'to_node_id': link_tmc.loc[i,'to_node_id'], \
                                            'time_period': reading_dict_selected['measurement_tstamp'][j][11:13]+\
                                                reading_dict_selected['measurement_tstamp'][j][14:16]+'_'+\
                                                reading_dict_selected['measurement_tstamp'][j+1][11:13]+\
                                                reading_dict_selected['measurement_tstamp'][j+1][14:16],\
                                            'date': reading_dict_selected['measurement_tstamp'][j][:10],\
                                            'geometry': link_tmc['geometry'][i],\
                                            'volume': None,\
                                            'travel_time': None,\
                                            'speed':round(np.mean(reading_dict_selected['speed'][j:j+1])),\
                                            'reference_speed': None,\
                                            'density': None,\
                                            'queue': None,\
                                            'notes': None }
            k += 1

    except:
        measurement_tmc_dict[k] = {'link_id_tmc': i,\
                                        'corridor_id': link_tmc['corridor_id'][i],\
                                        'corridor_link_order' : link_tmc['corridor_link_order'][i],\
                                        'from_node_id': link_tmc.loc[i,'from_node_id'], \
                                        'to_node_id': link_tmc.loc[i,'to_node_id'], \
                                        'time_period': None,\
                                        'date': None,\
                                        'geometry': link_tmc['geometry'][i],\
                                        'volume': None,\
                                        'travel_time': None,\
                                        'speed': None,\
                                        'reference_speed': None,\
                                        'density': None,\
                                        'queue': None,\
                                        'notes': None }
        k += 1

    if link_tmc.index.get_loc(i) > p/10 * len(link_tmc): 
        print(str(p*10)+"%"+' measurement_tmc completed!')
        p = p + 1

measurement_tmc = pd.DataFrame(measurement_tmc_dict).transpose()
measurement_tmc = measurement_tmc.dropna(subset=['time_period']) #remove na at the end of day of unrecorded ones
measurement_tmc = measurement_tmc.reset_index()
measurement_tmc = measurement_tmc.drop(['index'], 1)
measurement_tmc.to_csv('measurement_tmc.csv',index = False)
print('measurement_tmc.csv generated!')

10% measurement_tmc completed!
20% measurement_tmc completed!
30% measurement_tmc completed!
40% measurement_tmc completed!
50% measurement_tmc completed!
60% measurement_tmc completed!
70% measurement_tmc completed!
80% measurement_tmc completed!
90% measurement_tmc completed!
measurement_tmc.csv generated!


In [9]:
measurement_tmc

,link_id_tmc,corridor_id,corridor_link_order,from_node_id,to_node_id,time_period,date,geometry,volume,travel_time,speed,reference_speed,density,queue,notes
0,100000001,VA-7_EASTBOUND,38,100000001,100000002,0000_0005,2015-04-01,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",None,None,44,None,None,None,None
1,100000001,VA-7_EASTBOUND,38,100000001,100000002,0005_0010,2015-04-01,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",None,None,44,None,None,None,None
2,100000001,VA-7_EASTBOUND,38,100000001,100000002,0010_0015,2015-04-01,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",None,None,44,None,None,None,None
3,100000001,VA-7_EASTBOUND,38,100000001,100000002,0015_0020,2015-04-01,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",None,None,44,None,None,None,None
4,100000001,VA-7_EASTBOUND,38,100000001,100000002,0020_0025,2015-04-01,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",None,None,44,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4937,100000061,VA-7_WESTBOUND,48,100000062,100000063,0630_0635,2015-04-01,"LINESTRING (-77.3673 39.0125,-77.3707 39.0143)",None,None,47,None,None,None,None
4938,100000061,VA-7_WESTBOUND,48,100000062,100000063,0635_0640,2015-04-01,"LINESTRING (-77.3673 39.0125,-77.3707 39.0143)",None,None,58,None,None,None,None
4939,100000061,VA-7_WESTBOUND,48,100000062,100000063,0640_0645,2015-04-01,"LINESTRING (-77.3673 39.0125,-77.3707 39.0143)",None,None,60,None,None,None,None
4940,100000061,VA-7_WESTBOUND,48,100000062,100000063,0645_0650,2015-04-01,"LINESTRING (-77.3673 39.0125,-77.3707 39.0143)",None,None,57,None,None,None,None


## step 3 mapping osm network to tmc network

In [10]:
link_base = pd.read_csv('link.csv', low_memory=False)
link_base = link_base[link_base['link_type_name'].isin(['motorway','trunk'])]
# link_base = link_base[link_base['link_type_name'].isin(['motorway','trunk','primary','secondary'])]
link_base = link_base.reset_index()
link_base = link_base.drop(['index'], 1)

In [11]:
link_base

,name,link_id,osm_way_id,from_node_id,to_node_id,dir_flag,length,lanes,free_speed,capacity,link_type_name,link_type,geometry,allowed_uses,from_biway
0,NaN,28,5966229,9582,14365,1,149.790510,1,59,NaN,motorway,1,"LINESTRING (-77.1825104 38.9725558, -77.182093...",auto,0
1,NaN,821,8798183,11246,11513,1,279.776834,2,59,NaN,motorway,1,"LINESTRING (-77.2213136 38.8814293, -77.221272...",auto,0
2,NaN,826,8798251,471,13287,1,445.830075,4,59,NaN,motorway,1,"LINESTRING (-77.22733719999999 38.9345484, -77...",auto,0
3,NaN,827,8798256,843,14755,1,36.480261,1,59,NaN,motorway,1,"LINESTRING (-77.23355050000001 38.9330702, -77...",auto,0
4,NaN,844,8798416,9589,9593,1,527.119662,1,59,NaN,motorway,1,"LINESTRING (-77.18527570000001 38.9630504, -77...",auto,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1371,Dulles Toll Road,31115,939970508,846,845,1,146.592700,4,55,NaN,motorway,1,"LINESTRING (-77.2424048 38.9344861, -77.241861...",auto,0
1372,495 Express Lanes,31116,939986885,14357,11637,1,97.950079,3,65,NaN,motorway,1,"LINESTRING (-77.21353689999999 38.9265648, -77...",auto,0
1373,NaN,31117,940347578,14358,14359,1,107.136692,4,59,NaN,motorway,1,"LINESTRING (-77.26612110000001 38.8795003, -77...",auto,0
1374,NaN,31118,940347578,14359,14295,1,7.335470,4,59,NaN,motorway,1,"LINESTRING (-77.2672544 38.8798345, -77.267336...",auto,0


In [12]:
def LLs2Dist(lon1, lat1, lon2, lat2): #WGS84 transfer coordinate system to distance(meter) #xy #credit to xtHuang0927
    R = 6371
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0

    a = math.sin(dLat / 2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180.0) * math.cos(lat2 * math.pi / 180.0) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = R * c * 1000
    return distance

In [13]:
def getDegree(latA, lonA, latB, lonB):
    """
    Args:
        point p1(latA, lonA)
        point p2(latB, lonB)
    Returns:
        bearing between the two GPS points,
        default: the basis of heading direction is north
    """
    radLatA = math.radians(latA)
    radLonA = math.radians(lonA)
    radLatB = math.radians(latB)
    radLonB = math.radians(lonB)
    dLon = radLonB - radLonA
    y = math.sin(dLon) * math.cos(radLatB)
    x = math.cos(radLatA) * math.sin(radLatB) - math.sin(radLatA) * math.cos(radLatB) * math.cos(dLon)
    brng = np.degrees(math.atan2(y, x))
    brng = (brng + 360) % 360
    # brng = 360 - brng
    return brng
    

In [14]:
link_tmc

,name,corridor_id,corridor_link_order,from_node_id,to_node_id,directed,geometry_id,geometry,dir_flag,length,grade,facility_type,capacity,free_speed,lanes
link_id,,,,,,,,,,,,,,,
100000001,110N05711,VA-7_EASTBOUND,38.0,100000001,100000002,1,None,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",1,0.038620,None,None,None,None,None
100000002,110-05710,VA-7_EASTBOUND,39.0,100000002,100000003,1,None,"LINESTRING (-77.3709 39.0141,-77.3684 39.0127)",1,0.164895,None,None,None,None,None
100000003,110N05710,VA-7_EASTBOUND,40.0,100000003,100000004,1,None,"LINESTRING (-77.3684 39.0127,-77.3605 39.0086)",1,0.512158,None,None,None,None,None
100000004,110-05709,VA-7_EASTBOUND,41.0,100000004,100000005,1,None,"LINESTRING (-77.3605 39.0086,-77.3502 39.0028)",1,0.682589,None,None,None,None,None
100000005,110-05708,VA-7_EASTBOUND,42.0,100000005,100000006,1,None,"LINESTRING (-77.3502 39.0028,-77.3455 39.0005)",1,0.296731,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100000058,110+05709,VA-7_WESTBOUND,45.0,100000059,100000060,1,None,"LINESTRING (-77.346 39.001,-77.35 39.003)",1,0.254799,None,None,None,None,None
100000059,110+05710,VA-7_WESTBOUND,46.0,100000060,100000061,1,None,"LINESTRING (-77.35 39.003,-77.3609 39.009)",1,0.718593,None,None,None,None,None
100000060,110P05710,VA-7_WESTBOUND,47.0,100000061,100000062,1,None,"LINESTRING (-77.3609 39.009,-77.3673 39.0125)",1,0.421999,None,None,None,None,None


In [15]:
link_base

,name,link_id,osm_way_id,from_node_id,to_node_id,dir_flag,length,lanes,free_speed,capacity,link_type_name,link_type,geometry,allowed_uses,from_biway
0,NaN,28,5966229,9582,14365,1,149.790510,1,59,NaN,motorway,1,"LINESTRING (-77.1825104 38.9725558, -77.182093...",auto,0
1,NaN,821,8798183,11246,11513,1,279.776834,2,59,NaN,motorway,1,"LINESTRING (-77.2213136 38.8814293, -77.221272...",auto,0
2,NaN,826,8798251,471,13287,1,445.830075,4,59,NaN,motorway,1,"LINESTRING (-77.22733719999999 38.9345484, -77...",auto,0
3,NaN,827,8798256,843,14755,1,36.480261,1,59,NaN,motorway,1,"LINESTRING (-77.23355050000001 38.9330702, -77...",auto,0
4,NaN,844,8798416,9589,9593,1,527.119662,1,59,NaN,motorway,1,"LINESTRING (-77.18527570000001 38.9630504, -77...",auto,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1371,Dulles Toll Road,31115,939970508,846,845,1,146.592700,4,55,NaN,motorway,1,"LINESTRING (-77.2424048 38.9344861, -77.241861...",auto,0
1372,495 Express Lanes,31116,939986885,14357,11637,1,97.950079,3,65,NaN,motorway,1,"LINESTRING (-77.21353689999999 38.9265648, -77...",auto,0
1373,NaN,31117,940347578,14358,14359,1,107.136692,4,59,NaN,motorway,1,"LINESTRING (-77.26612110000001 38.8795003, -77...",auto,0
1374,NaN,31118,940347578,14359,14295,1,7.335470,4,59,NaN,motorway,1,"LINESTRING (-77.2672544 38.8798345, -77.267336...",auto,0


In [22]:
multiline_string_tmc_list = []
multiline_string_tmc_list_sub = []
for j in link_tmc.index:
    link_tmc_geometry_list = link_tmc.loc[j,'geometry'][12:-1].split(",")
    for link_tmc_geometry in link_tmc_geometry_list:
        multiline_string_tmc_list_sub.append((float(link_tmc_geometry.split(" ")[0]),float(link_tmc_geometry.split(" ")[1])))
    multiline_string_tmc_list_sub = tuple(multiline_string_tmc_list_sub)
    multiline_string_tmc_list.append(multiline_string_tmc_list_sub)
    multiline_string_tmc_list_sub = []

In [26]:
multiline_string_base_list = []
multiline_string_base_list_sub = []
for j in link_base.index:
    link_base_geometry_list = link_base.loc[j,'geometry'][12:-1].split(", ")
    for link_base_geometry in link_base_geometry_list:
        multiline_string_base_list_sub.append((float(link_base_geometry.split(" ")[0]),float(link_base_geometry.split(" ")[1])))
    multiline_string_base_list_sub = tuple(multiline_string_base_list_sub)
    multiline_string_base_list.append(multiline_string_base_list_sub)
    multiline_string_base_list_sub = []

In [30]:
from shapely.geometry import MultiLineString

line_tmc = MultiLineString(multiline_string_tmc_list)
line_base = MultiLineString(multiline_string_base_list)
line_tmc.overlaps(line_base)

False

In [40]:
print(line_tmc.bounds[0] > line_base.bounds[0])
print(line_tmc.bounds[1] > line_base.bounds[1])
print(line_tmc.bounds[2] > line_base.bounds[2])
print(line_tmc.bounds[3] > line_base.bounds[3])

True
True
False
False


In [16]:
# if 
matching_tmc2gmns_dict = {}
k = 0
p = 1

for j in link_tmc.index:
    lon_tmc_list = []
    lat_tmc_list = []
    link_tmc_geometry_list = link_tmc.loc[j,'geometry'][12:-1].split(",")
    for link_tmc_geometry in link_tmc_geometry_list:
        lon_tmc_list.append(float(link_tmc_geometry.split(" ")[0]))
        lat_tmc_list.append(float(link_tmc_geometry.split(" ")[1]))
    center_tmc_lon = np.mean(lon_tmc_list)
    center_tmc_lat = np.mean(lat_tmc_list)
    tmc_lon_1 = lon_tmc_list[0]
    tmc_lon_2 = lon_tmc_list[-1]
    tmc_lat_1 = lat_tmc_list[0]
    tmc_lat_2 = lat_tmc_list[-1]
    if getDegree(tmc_lat_1,tmc_lon_1,tmc_lat_2,tmc_lon_2)>180:
        angle_tmc = getDegree(tmc_lat_2,tmc_lon_2,tmc_lat_1,tmc_lon_1)
    else:
        angle_tmc = getDegree(tmc_lat_1,tmc_lon_1,tmc_lat_2,tmc_lon_2)


    distance_list = []
    angle_list = []
    for i in range(len(link_base)):
        lon_list = []
        lat_list = [] 
        link_geometry_list = link_base.loc[i,'geometry'][12:-1].split(", ")
        for link_geometry in link_geometry_list:
            lon_list.append(float(link_geometry.split(" ")[0]))
            lat_list.append(float(link_geometry.split(" ")[1]))
        '''distance'''
        center_lon = np.mean(lon_list)
        center_lat = np.mean(lat_list)
        distance_list.append(LLs2Dist(center_lon, center_lat, center_tmc_lon, center_tmc_lat))
        '''angle '''
        base_lon_1 = lon_list[0]
        base_lon_2 = lon_list[-1]
        base_lat_1 = lat_list[0]
        base_lat_2 = lat_list[-1]
        if getDegree(base_lat_1,base_lon_1,base_lat_2,base_lon_2)>180:
            angle_base = getDegree(base_lat_2,base_lon_2,base_lat_1,base_lon_1)
        else:
            angle_base = getDegree(base_lat_1,base_lon_1,base_lat_2,base_lon_2)
        
        if abs(angle_tmc - angle_base) >= 90:
            relative_angle = 180 - abs(angle_tmc - angle_base)
        else:
            relative_angle = abs(angle_tmc - angle_base)
        angle_list.append(relative_angle)

    small_angle_list = [i for i, value in enumerate(angle_list) if value < 45]
    df_distance = pd.DataFrame({'distance':distance_list})
    
    nearest_index = df_distance.loc[small_angle_list].idxmin().values[0]

    matching_tmc2gmns_dict[k] = {'name_tmc':link_tmc.loc[j]['name'],\
                                'corridor_id_tmc':link_tmc.loc[j]['corridor_id'],\
                                'link_id_tmc':link_tmc.loc[[j]].index.values[0],\
                                'from_node_id_tmc':link_tmc.loc[j]['from_node_id'],\
                                'to_node_id_tmc':link_tmc.loc[j]['to_node_id'],\
                                'category_id_tmc':link_tmc.index.get_loc(j)+1,\
                                'geometry_tmc':link_tmc.loc[j]['geometry'],\
                                'name_base':link_base['name'][nearest_index],\
                                'link_id_base':link_base['link_id'][nearest_index],\
                                'from_node_id_base':link_base['from_node_id'][nearest_index],\
                                'to_node_id_base':link_base['to_node_id'][nearest_index],\
                                'category_id_base':link_tmc.index.get_loc(j)+1,\
                                'geometry_base':link_base['geometry'][nearest_index],\
                                'distance':min(distance_list),\
                                'geometry_tmc_base':'MULTILINESTRING ('+ link_tmc.loc[j]['geometry'][11:] + \
                                                    ', ' + link_base['geometry'][nearest_index][11:]+')'}
    k += 1

    
    if link_tmc.index.get_loc(j) > p/10 * len(link_tmc): 
        print(str(p*10)+"%"+' matching completed!')
        p = p + 1
        

matching_tmc2gmns = pd.DataFrame(matching_tmc2gmns_dict).transpose()

matching_tmc2gmns.to_csv('matching_tmc2gmns.csv',index = False)
print('matching_tmc2gmns.csv generated!')

10% matching completed!
20% matching completed!
30% matching completed!
40% matching completed!
50% matching completed!
60% matching completed!
70% matching completed!
80% matching completed!
90% matching completed!
matching_tmc2gmns.csv generated!


In [17]:
matching_tmc2gmns

,name_tmc,corridor_id_tmc,link_id_tmc,from_node_id_tmc,to_node_id_tmc,category_id_tmc,geometry_tmc,name_base,link_id_base,from_node_id_base,to_node_id_base,category_id_base,geometry_base,distance,geometry_tmc_base
0,110N05711,VA-7_EASTBOUND,100000001,100000001,100000002,1,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",Harry Byrd Highway,30827,9621,14870,1,"LINESTRING (-77.37181750000001 39.0145561, -77...",6.33785,"MULTILINESTRING ((-77.3715 39.0144,-77.3709 39..."
1,110-05710,VA-7_EASTBOUND,100000002,100000002,100000003,2,"LINESTRING (-77.3709 39.0141,-77.3684 39.0127)",Leesburg Pike,18653,14870,9642,2,"LINESTRING (-77.37091289999999 39.0140956, -77...",48.6156,"MULTILINESTRING ((-77.3709 39.0141,-77.3684 39..."
2,110N05710,VA-7_EASTBOUND,100000003,100000003,100000004,3,"LINESTRING (-77.3684 39.0127,-77.3605 39.0086)",NaN,23889,9961,9569,3,"LINESTRING (-77.3644748 39.0102989, -77.364278...",29.5026,"MULTILINESTRING ((-77.3684 39.0127,-77.3605 39..."
3,110-05709,VA-7_EASTBOUND,100000004,100000004,100000005,4,"LINESTRING (-77.3605 39.0086,-77.3502 39.0028)",Leesburg Pike,28010,13667,2478,4,"LINESTRING (-77.3552028 39.0059016, -77.355703...",39.0031,"MULTILINESTRING ((-77.3605 39.0086,-77.3502 39..."
4,110-05708,VA-7_EASTBOUND,100000005,100000005,100000006,5,"LINESTRING (-77.3502 39.0028,-77.3455 39.0005)",Leesburg Pike,23851,9988,9989,5,"LINESTRING (-77.3493028 39.0024295, -77.347737...",111.754,"MULTILINESTRING ((-77.3502 39.0028,-77.3455 39..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,110+05709,VA-7_WESTBOUND,100000058,100000059,100000060,58,"LINESTRING (-77.346 39.001,-77.35 39.003)",Leesburg Pike,23851,9988,9989,58,"LINESTRING (-77.3493028 39.0024295, -77.347737...",144.799,"MULTILINESTRING ((-77.346 39.001,-77.35 39.003..."
58,110+05710,VA-7_WESTBOUND,100000059,100000060,100000061,59,"LINESTRING (-77.35 39.003,-77.3609 39.009)",Leesburg Pike,28010,13667,2478,59,"LINESTRING (-77.3552028 39.0059016, -77.355703...",4.62211,"MULTILINESTRING ((-77.35 39.003,-77.3609 39.00..."
59,110P05710,VA-7_WESTBOUND,100000060,100000061,100000062,60,"LINESTRING (-77.3609 39.009,-77.3673 39.0125)",NaN,23893,9960,9602,60,"LINESTRING (-77.3634677 39.0110608, -77.363677...",3.30337,"MULTILINESTRING ((-77.3609 39.009,-77.3673 39...."
60,110+05711,VA-7_WESTBOUND,100000061,100000062,100000063,61,"LINESTRING (-77.3673 39.0125,-77.3707 39.0143)",Leesburg Pike,19077,10236,10237,61,"LINESTRING (-77.3684813 39.0131606, -77.368831...",34.1158,"MULTILINESTRING ((-77.3673 39.0125,-77.3707 39..."


## step 4 generating measurement_osm 

In [18]:
link_base = pd.read_csv('link.csv', low_memory=False)
link_base = link_base[link_base['link_type_name'].isin(['motorway','trunk','primary','secondary'])]
link_base = link_base.reset_index()
link_base = link_base.drop(['index'], 1)

'''build measurement_base.csv''' 
measurement_base = pd.DataFrame()
measurement_base['link_id'] = None
measurement_base['osm_way_id'] = None
measurement_base['from_node_id'] = None
measurement_base['to_node_id'] = None
measurement_base['lanes'] = None
measurement_base['length'] = None
measurement_base['time_period'] = None
measurement_base['date'] = None
measurement_base['geometry'] = None
measurement_base['volume'] = None
measurement_base['speed'] = None
measurement_base['ip_address'] = None

k=0
p=1
measurement_base_dict = {}
for i in matching_tmc2gmns.index:
    try:
        measurement_tmc_selected = measurement_tmc[measurement_tmc['link_id_tmc'] == matching_tmc2gmns['link_id_tmc'][i]]
        link_base_selected = link_base[link_base['link_id'] == matching_tmc2gmns['link_id_base'][i]]
        for j in measurement_tmc_selected.index:
            measurement_base_dict[k] = {'link_id': link_base_selected['link_id'].values[0],\
                                            'osm_way_id':link_base_selected['osm_way_id'].values[0],\
                                            'from_node_id': link_base_selected['from_node_id'].values[0],\
                                            'to_node_id': link_base_selected['to_node_id'].values[0],\
                                            'lanes': link_base_selected['lanes'].values[0], \
                                            'length': link_base_selected['length'].values[0], \
                                            'link_type_name': link_base_selected['link_type_name'].values[0], \
                                            'time_period': measurement_tmc_selected['time_period'][j],\
                                            'date': measurement_tmc_selected['date'][j],\
                                            'geometry': link_base_selected['geometry'].values[0],\
                                            'volume': None,\
                                            'speed': measurement_tmc_selected['speed'][j],\
                                            'ip_address': 'www.openstreetmap.org/?way=' + str(link_base_selected['osm_way_id'].values[0])} 

            k += 1
    except:
        measurement_base_dict[k] = {'link_id': link_base_selected['link_id'].values[0],\
                                            'osm_way_id':link_base_selected['osm_way_id'].values[0],\
                                            'from_node_id': link_base_selected['from_node_id'].values[0],\
                                            'to_node_id': link_base_selected['to_node_id'].values[0],\
                                            'lanes': link_base_selected['lanes'].values[0], \
                                            'length': link_base_selected['length'].values[0], \
                                            'link_type_name': link_base_selected['link_type_name'].values[0], \
                                            'time_period':None,\
                                            'date': None,\
                                            'geometry': link_base_selected['geometry'].values[0],\
                                            'volume': None,\
                                            'speed': None,\
                                            'ip_address': 'www.openstreetmap.org/?way=' + str(link_base_selected['osm_way_id'].values[0])}

        k += 1
    
    if i+1 > p/10 * len(matching_tmc2gmns.index): 
        print(str(p*10)+"%"+' measurement_base completed!')
        p = p + 1

measurement_base = pd.DataFrame(measurement_base_dict).transpose()
measurement_base.to_csv('measurement_base.csv',index = False)
print('measurement_base.csv generated!')

10% measurement_base completed!
20% measurement_base completed!
30% measurement_base completed!
40% measurement_base completed!
50% measurement_base completed!
60% measurement_base completed!
70% measurement_base completed!
80% measurement_base completed!
90% measurement_base completed!
measurement_base.csv generated!


In [19]:
measurement_base

,link_id,osm_way_id,from_node_id,to_node_id,lanes,length,link_type_name,time_period,date,geometry,volume,speed,ip_address
0,30827,879886802,9621,14870,3,78.6529,trunk,0000_0005,2015-04-01,"LINESTRING (-77.37181750000001 39.0145561, -77...",None,44,www.openstreetmap.org/?way=879886802
1,30827,879886802,9621,14870,3,78.6529,trunk,0005_0010,2015-04-01,"LINESTRING (-77.37181750000001 39.0145561, -77...",None,44,www.openstreetmap.org/?way=879886802
2,30827,879886802,9621,14870,3,78.6529,trunk,0010_0015,2015-04-01,"LINESTRING (-77.37181750000001 39.0145561, -77...",None,44,www.openstreetmap.org/?way=879886802
3,30827,879886802,9621,14870,3,78.6529,trunk,0015_0020,2015-04-01,"LINESTRING (-77.37181750000001 39.0145561, -77...",None,44,www.openstreetmap.org/?way=879886802
4,30827,879886802,9621,14870,3,78.6529,trunk,0020_0025,2015-04-01,"LINESTRING (-77.37181750000001 39.0145561, -77...",None,44,www.openstreetmap.org/?way=879886802
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4937,19077,59763019,10236,10237,3,36.0596,trunk,0630_0635,2015-04-01,"LINESTRING (-77.3684813 39.0131606, -77.368831...",None,47,www.openstreetmap.org/?way=59763019
4938,19077,59763019,10236,10237,3,36.0596,trunk,0635_0640,2015-04-01,"LINESTRING (-77.3684813 39.0131606, -77.368831...",None,58,www.openstreetmap.org/?way=59763019
4939,19077,59763019,10236,10237,3,36.0596,trunk,0640_0645,2015-04-01,"LINESTRING (-77.3684813 39.0131606, -77.368831...",None,60,www.openstreetmap.org/?way=59763019
4940,19077,59763019,10236,10237,3,36.0596,trunk,0645_0650,2015-04-01,"LINESTRING (-77.3684813 39.0131606, -77.368831...",None,57,www.openstreetmap.org/?way=59763019


In [19]:
print( __name__)
print(os.path.dirname(os.path.realpath('__file__')))
print(os.path.realpath('__file__'))
print(os.getcwd())

__main__
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS/__file__
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS
